In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp lag_transforms

# Lag transforms
> Built-in lag transformations

In [ ]:
#| export
from typing import Optional

import numpy as np
try:
    import coreforecast.lag_transforms as core_tfms
    from coreforecast.grouped_array import GroupedArray as CoreGroupedArray
except ImportError:
    raise ImportError(
        'The lag_transforms module requires the coreforecast package. '
        'Please install it with `pip install coreforecast`.\n'
        'You can also install mlforecast with the lag_transforms extra: `pip install "mlforecast[lag_transforms]"`'
    ) from None
from sklearn.base import BaseEstimator

In [ ]:
#| exporti
class BaseLagTransform(BaseEstimator):
    _core_tfm: core_tfms.BaseLagTransform

    def transform(self, ga: CoreGroupedArray) -> np.ndarray:
        return self._core_tfm.transform(ga)

    def update(self, ga: CoreGroupedArray) -> np.ndarray:
        return self._core_tfm.update(ga)

In [ ]:
#| exporti
class Lag(BaseLagTransform):
    def __init__(self, lag: int):
        self.lag = lag
        self._core_tfm = core_tfms.Lag(lag=lag)

    def __eq__(self, other):
        return isinstance(other, Lag) and self.lag == other.lag

In [ ]:
#| exporti
class RollingBase(BaseLagTransform):
    "Rolling statistic"
    def __init__(self, window_size: int, min_samples: Optional[int] = None):
        """
        Parameters
        ----------
        window_size : int
            Number of samples in the window.
        min_samples: int
            Minimum samples required to output the statistic.
            If `None`, will be set to `window_size`.
        """
        self.window_size = window_size
        self.min_samples = min_samples

    def _set_core_tfm(self, lag: int):
        self._core_tfm = getattr(core_tfms, self.tfm_name)(lag=lag, window_size=self.window_size, min_samples=self.min_samples)
        return self

In [ ]:
#| export
class RollingMean(RollingBase):
    tfm_name = 'RollingMean'

class RollingStd(RollingBase):
    tfm_name = 'Rollingstd'


class RollingMin(RollingBase):
    tfm_name = "RollingMin"


class RollingMax(RollingBase):
    tfm_name = "RollingMax"

In [ ]:
#| hide
rng = np.random.default_rng(seed=0)
lengths = rng.integers(low=50, high=100, size=20)
data = rng.random(lengths.sum())
ga = CoreGroupedArray(data, np.append(0, lengths.cumsum()))
RollingMean(7)._set_core_tfm(1).transform(ga)

array([       nan,        nan,        nan, ..., 0.32114229, 0.3672723 ,
       0.39137066])

In [ ]:
#| exporti
class SeasonalRollingBase(BaseLagTransform):
    """Rolling statistic over seasonal periods"""
    def __init__(
        self, season_length: int, window_size: int, min_samples: Optional[int] = None
    ):
        """
        Parameters
        ----------
        season_length : int
            Periodicity of the seasonal period.
        window_size : int
            Number of samples in the window.
        min_samples: int
            Minimum samples required to output the statistic.
            If `None`, will be set to `window_size`.
        """        
        self.season_length = season_length
        self.window_size = window_size
        self.min_samples = min_samples

    def _set_core_tfm(self, lag: int):
        self._core_tfm = getattr(core_tfms, self.tfm_name)(
            lag=lag, season_length=self.season_length, window_size=self.window_size, min_samples=self.min_samples
        )
        return self

In [ ]:
# | export
class SeasonalRollingMean(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingMean'

class SeasonalRollingStd(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingStd'

class SeasonalRollingMin(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingMin'

class SeasonalRollingMax(SeasonalRollingBase):
    tfm_name = 'SeasonalRollingMax'

In [ ]:
#| hide
SeasonalRollingStd(7, 4)._set_core_tfm(2).transform(ga)

array([       nan,        nan,        nan, ..., 0.35518094, 0.25199008,
       0.40335074])

In [ ]:
#| exporti
class ExpandingBase(BaseLagTransform):
    """Expanding statistic"""
    def __init__(self):
        ...
    
    def _set_core_tfm(self, lag: int):
        self._core_tfm = getattr(core_tfms, self.tfm_name)(lag=lag)
        return self

In [ ]:
#| export
class ExpandingMean(ExpandingBase):
    tfm_name = 'ExpandingMean'

class ExpandingStd(ExpandingBase):
    tfm_name = 'ExpandingStd'

class ExpandingMin(ExpandingBase):
    tfm_name = 'ExpandingMin'

class ExpandingMax(ExpandingBase):
    tfm_name = 'ExpandingMax'

In [ ]:
#| hide
ExpandingMin()._set_core_tfm(3).transform(ga)

array([       nan,        nan,        nan, ..., 0.00297614, 0.00297614,
       0.00297614])

In [ ]:
#| export
class ExponentiallyWeightedMean(BaseLagTransform):
    """Exponentially weighted average"""
    def __init__(self, alpha: float):
        """
        Parameters
        ----------
        alpha : float
            Smoothing factor.
        """
        self.alpha = alpha

    def _set_core_tfm(self, lag: int):        
        self._core_tfm = core_tfms.ExponentiallyWeightedMean(lag=lag, alpha=self.alpha)
        return self

In [ ]:
#| hide
ExponentiallyWeightedMean(0.7)._set_core_tfm(4).transform(ga)

array([       nan,        nan,        nan, ..., 0.3074053 , 0.5567787 ,
       0.31390901])